In [1]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.4 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import psycopg2
import json
import os
from urllib.parse import quote
from datetime import datetime
from dotenv import load_dotenv

In [3]:
TABLE_SUFFIX = "ELN_WRITEUP_INTERP_DATA"
DS_DATA = {'dev' : {'ds_id': 1751, 'domain': 'prelude-dev', 'project_id': 104000},
           'up6' : {'ds_id': 1372, 'domain': 'prelude-upgrade6', 'project_id': 96000},
           'prod' : {'ds_id': 1419, 'domain': 'prelude', 'project_id': 99000},
          }
BASE_URL = "dotmatics.net"
ISID = "preludeAdmin"
PASSWORD = quote(os.getenv('DM_PASS'))
EXPIRATION = 4*60*60
formats = ["%Y-%m-%d", "%d/%m/%Y"]

In [13]:
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME")

In [5]:
def create_table_if_not_exists(cursor, table_name):
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
      id SERIAL PRIMARY KEY,
      entry_date DATE,
      experiment_id VARCHAR(100),
      match_position INT,
      mask_id VARCHAR(400),
      unique_id VARCHAR(150),
      mask_title VARCHAR(200),
      mask_text VARCHAR(300),
      write_up TEXT
    );
    """
    cursor.execute(create_table_query)

def parse_date(date_str):
    """Parse date string into consistent 'YYYY-MM-DD' format."""
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d")
        except ValueError:
            continue
    raise ValueError(f"Invalid date format: {date_str}")

def insert_data(cursor, table_name, entry_date, experiment_id, match_position, mask_id, unique_id, mask_title, mask_text, write_up):
    insert_query = f"""
        INSERT INTO {table_name} (entry_date, experiment_id, match_position, mask_id, unique_id, mask_title, mask_text, write_up)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (entry_date, experiment_id, match_position, mask_id, unique_id, mask_title, mask_text, write_up))

def process_response_and_save_to_db(response_json, cursor, table_name):
    for experiment_id, details in response_json.items():
        data_sources = details.get("dataSources", {})
        for source_id, entries in data_sources.items():
            for entry in entries.values():
                insert_data(
                    cursor,
                    table_name,
                    parse_date(entry.get("ENTRY_DATE")),
                    experiment_id,
                    entry.get("MATCH_POSITION"),
                    entry.get("MASK_ID"),
                    entry.get("UNIQUE_ID"),
                    entry.get("MASK_TITLE"),
                    entry.get("MASK_TEXT"),
                    entry.get("WRITE_UP")
                )

def process_experiment_ids(file_path, db_connection, DOMAIN, DS_ID, PROJECT_ID, API_TOKEN):
    table_name = f"{DOMAIN.replace('prelude-', '').replace('prelude', 'prod').upper()}_{TABLE_SUFFIX}"

    with db_connection.cursor() as cursor:
        create_table_if_not_exists(cursor, table_name)

        with open(file_path, "r") as file:
            for line in file:
                ids_str = line.strip()
                api_url = f"https://{DOMAIN}.{BASE_URL}/browser/api/data/{ISID}/{PROJECT_ID}/{DS_ID}/{ids_str}?token={API_TOKEN}"
                print(f'fetching data - {api_url}')
                print()
                response = requests.get(api_url)
                if response.status_code == 200:
                    data = response.json()
                    process_response_and_save_to_db(data, cursor, table_name)
                    empty_exp_ids = [key for key, item in data.items() if not item['dataSources']]
                    if empty_exp_ids:
                        print(f'# empty exp ids: {len(empty_exp_ids)} - {empty_exp_ids}')
                        print()
                else:
                    print(f"Failed to fetch data for IDs in line: {line.strip()[:50]}...: {response.status_code}")
            
        db_connection.commit()

In [6]:
# def fetch_data(DOMAIN, DS_ID, PROJECT_ID, API_TOKEN):
#     """
#     for testing purposes; no sql data upload; uses wildcard selection
#     """
#     table_name = f"{DOMAIN.replace('-', '_').upper()}_{TABLE_SUFFIX}"
#     api_url = f"https://{DOMAIN}.{BASE_URL}/browser/api/data/{ISID}/{PROJECT_ID}/{DS_ID}/*?token={API_TOKEN}"
#     print(f'fetching data - {api_url}')
#     print()
#     response = requests.get(api_url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         print(f"Failed to fetch data for IDs in line: {line.strip()[:50]}...: {response.status_code}")

In [7]:
# for testing purposes, no datbase upload
# ds_name = 'up6'
# api_token = retrieve_token(DS_DATA[ds_name]['domain'])
# print(api_token)
# try:
#     response = fetch_data(DS_DATA[ds_name]['domain'], DS_DATA[ds_name]['ds_id'], DS_DATA[ds_name]['project_id'], api_token)
# except Exception as e:
#     print(f"Error: {e}")

In [8]:
# len(response)

In [9]:
# for testing purposes, no datbase upload, use of wildcard
# empty_count = sum(1 for item in response.values() if not item['dataSources'])
# empty_count

In [10]:
def main(file_path, domain, ds_id, proj_id, api_token):
    if not os.path.exists(file_path):
        print("File exp_ids.txt not found!")
        return
    
    try:
        db_connection = psycopg2.connect(
            host=DB_HOST, 
            user=DB_USER, 
            password=DB_PASSWORD,
            dbname=DB_NAME
        )
        process_experiment_ids(file_path, db_connection, domain, ds_id, proj_id, api_token)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if db_connection:
            db_connection.close()

In [11]:
def retrieve_token(DOMAIN):
    token_url = f"https://{DOMAIN}.{BASE_URL}/browser/api/authenticate/requestToken?isid={ISID}&password={PASSWORD}&expiration={EXPIRATION}"
    # print(token_url)
    try:
        response = requests.get(token_url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to retrieve token: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error retrieving token: {e}")
        return None

### UPGRADE 6 interpolated data

In [ ]:
file_path = "/home/jovyan/work/Documents/RCH/exp_ids_eln_writeup_compr.txt"
ds_name = 'up6'
domain = DS_DATA[ds_name]['domain']
ds_id = DS_DATA[ds_name]['ds_id']
proj_id = DS_DATA[ds_name]['project_id']
api_token = retrieve_token(domain)
print(api_token)

main(file_path, domain, ds_id, proj_id, api_token)

### DEV - interpolated data 

In [ ]:
file_path = "/home/jovyan/work/Documents/RCH/exp_ids_eln_writeup_compr.txt"
ds_name = 'dev'
domain = DS_DATA[ds_name]['domain']
ds_id = DS_DATA[ds_name]['ds_id']
proj_id = DS_DATA[ds_name]['project_id']
api_token = retrieve_token(domain)
print(api_token)

main(file_path, domain, ds_id, proj_id, api_token)

### PROD interpolated data

In [ ]:
# PROD
file_path = "/home/jovyan/work/Documents/RCH/exp_ids_eln_writeup_compr.txt"
ds_name = 'prod'
domain = DS_DATA[ds_name]['domain']
ds_id = DS_DATA[ds_name]['ds_id']
proj_id = DS_DATA[ds_name]['project_id']
api_token = retrieve_token(domain)
print(api_token)

main(file_path, domain, ds_id, proj_id, api_token)

In [17]:
import pandas as pd
import re
from sqlalchemy import create_engine
from IPython.display import HTML, display

In [18]:
db_url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
engine = create_engine(db_url)

table_up6_query = "SELECT * FROM upgrade6_eln_writeup_interp_data"
table_dev_query = "SELECT * FROM dev_eln_writeup_interp_data"
table_prod_query = "SELECT * FROM prod_eln_writeup_interp_data"

df_up6 = pd.read_sql(table_up6_query, engine)
df_dev = pd.read_sql(table_dev_query, engine)
df_prod = pd.read_sql(table_prod_query, engine)
df_prod = df_prod.rename(columns=lambda x: f"{x}_prod" if x not in ["experiment_id", "match_position"] else x)
                                                                    
merged_df_temp = pd.merge(
  df_up6,
  df_dev,
  on=["experiment_id", "match_position"], 
  suffixes=("_u6", "_dev")
) 

merged_df = pd.merge(
  merged_df_temp,
  df_prod,
  on=["experiment_id", "match_position"]
)

merged_dff = merged_df[merged_df["match_position"] == 1]
merged_dff = merged_dff[['entry_date_prod', 'experiment_id', 'write_up_u6', 'write_up_dev', 'write_up_prod']]

In [19]:
filtered_df = merged_df[ 
    (merged_df['mask_id_u6'] != merged_df['mask_id_dev']) |
    (merged_df['unique_id_u6'] != merged_df['unique_id_dev']) |
    (merged_df['mask_title_u6'] != merged_df['mask_title_dev']) |
    (merged_df['mask_text_u6'] != merged_df['mask_text_dev']) |
    (merged_df['mask_id_u6'] != merged_df['mask_id_prod']) |
    (merged_df['unique_id_u6'] != merged_df['unique_id_prod']) |
    (merged_df['mask_title_u6'] != merged_df['mask_title_prod']) |
    (merged_df['mask_text_u6'] != merged_df['mask_text_prod']) 
]

In [82]:
# len(filtered_df)

1894

In [20]:
diff_columns = ['mask_id', 'unique_id', 'mask_title', 'mask_text']
ordered_columns = ["entry_date_dev", "experiment_id", "match_position"] 
for col in diff_columns:
    ordered_columns.append(f"{col}_u6")
    ordered_columns.append(f"{col}_dev") 
    ordered_columns.append(f"{col}_prod") 

# ordered_columns

In [21]:
result = filtered_df[ordered_columns]
# result

In [83]:
# result.to_csv("differences.csv", index=False)

In [103]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [130]:
# merged_df[(merged_df['experiment_id'] == '206974') ]

,id_u6,entry_date_u6,experiment_id,match_position,mask_id_u6,unique_id_u6,mask_title_u6,mask_text_u6,write_up_u6,id_dev,entry_date_dev,mask_id_dev,unique_id_dev,mask_title_dev,mask_text_dev,write_up_dev,id_prod,entry_date_prod,mask_id_prod,unique_id_prod,mask_title_prod,mask_text_prod,write_up_prod
0,1,2022-09-11,206974,3,3,uid 4450,solvent volume,{SOLVENT_NAME} ({VOLUME} {VOLUME_UNITS}),None,2,2022-09-11,3,uid 4450,solvent volume,{SOLVENT_NAME} ({VOLUME} {VOLUME_UNITS}),None,2,2022-09-11,3,uid 4450,solvent volume,{SOLVENT_NAME} ({VOLUME} {VOLUME_UNITS}),None
1,2,2022-09-11,206974,1,9,uid 2,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)","<p><span style=""font-size: 12pt;"">To a solution of&nbsp;<span style=""color: #0B3142"">1-(2-bromo-1,3-thiazol-5-yl)-2,2-difluoroethanone (?, .181 mmol)</span> &nbsp;and&nbsp;<span style=""color: #0B3142"">N-(1-methylsulfonylpiperidin-4-yl)-5-(trifluoromethyl)-4-trimethylstannylpyrimidin-2-amine (?, .181 mmol)</span> &nbsp;in&nbsp;<span style=""color: #53DD6C"">1,4-Dioxane (1.5 mL)</span> &nbsp;was added&nbsp;<span style=""color: #0B3142"">bis(triphenylphosphine)palladium(II) dichloride (?, .036 mmol)</span> &nbsp;,&nbsp;<span style=""color: #0B3142"">copper(I) iodide (?, .036 mmol)</span> &nbsp;and&nbsp;<span style=""color: #0B3142"">Lithium Chloride (?, .542 mmol)</span> . The reaction mixture was bubbled with N2 for 1min and stirred at&nbsp;<span style=""color: #EB5E28"">100 °C</span> for 3h. LCMS showed the SM was consumed. The reaciton was quencehd with KF solution and stirred at rt for 1h. The solution was filtered and concentrated under reduced pressure.&nbsp;The residue was purified by prep-HPLC on a C18 column eluting with MeCN/H2O (20-60%) with 0.1% TFA to yield&nbsp;<span style=""color: #6A7FDB"">2,2-difluoro-1-[2-[2-[(1-methylsulfonylpiperidin-4-yl)amino]-5-(trifluoromethyl)pyrimidin-4-yl]-1,3-thiazol-5-yl]ethanone (22 mg, .045 mmol, 25.087% yield)</span>&nbsp;</span></p>",5,2022-09-11,9,uid 2,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)","<p><span style=""font-size: 12pt;"">To a solution of&nbsp;<span style=""color: #0B3142"">1-(2-bromo-1,3-thiazol-5-yl)-2,2-difluoroethanone (?, .181 mmol)</span> &nbsp;and&nbsp;<span style=""color: #0B3142"">N-(1-methylsulfonylpiperidin-4-yl)-5-(trifluoromethyl)-4-trimethylstannylpyrimidin-2-amine (?, .181 mmol)</span> &nbsp;in&nbsp;<span style=""color: #53DD6C"">1,4-Dioxane (1.5 mL)</span> &nbsp;was added&nbsp;<span style=""color: #0B3142"">bis(triphenylphosphine)palladium(II) dichloride (?, .036 mmol)</span> &nbsp;,&nbsp;<span style=""color: #0B3142"">copper(I) iodide (?, .036 mmol)</span> &nbsp;and&nbsp;<span style=""color: #0B3142"">Lithium Chloride (?, .542 mmol)</span> . The reaction mixture was bubbled with N2 for 1min and stirred at&nbsp;<span style=""color: #EB5E28"">100 °C</span> for 3h. LCMS showed the SM was consumed. The reaciton was quencehd with KF solution and stirred at rt for 1h. The solution was filtered and concentrated under reduced pressure.&nbsp;The residue was purified by prep-HPLC on a C18 column eluting with MeCN/H2O (20-60%) with 0.1% TFA to yield&nbsp;<span style=""color: #6A7FDB"">2,2-difluoro-1-[2-[2-[(1-methylsulfonylpiperidin-4-yl)amino]-5-(trifluoromethyl)pyrimidin-4-yl]-1,3-thiazol-5-yl]ethanone (22 mg, .045 mmol, 25.087% yield)</span>&nbsp;</span></p>",5,2022-09-11,9,uid 2,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)","<p><span style=""font-size: 12pt;"">To a solution of&nbsp;<span style=""color: #0B3142"">1-(2-bromo-1,3-thiazol-5-yl)-2,2-difluoroethanone (?, .181 mmol)</span> &nbsp;and&nbsp;<span style=""color: #0B3142"">N-(1-methylsulfonylpiperidin-4-yl)-5-(trifluoromethyl)-4-trimethylstannylpyrimidin-2-amine (?, .181 mmol)</span> &nbsp;in&nbsp;<span style=""color: #53DD6C"">1,4-Dioxane (1.5 mL)</span> &nbsp;was added&nbsp;<span style=""color: #0B3142"">bis(triphenylphosphine)palladium(II) dichloride (?, .036 mmol)</span> &nbsp;,&nbsp;<span style=""col

In [35]:
merged_df[(merged_df['experiment_id'] == '207225')][['entry_date_u6', 'experiment_id', 'match_position', 'mask_id_u6', 'unique_id_u6', 'mask_title_u6', 'mask_text_u6', 'mask_id_dev', 'unique_id_dev', 'mask_title_dev', 'mask_text_dev', 'mask_id_prod', 'unique_id_prod', 'mask_title_prod', 'mask_text_prod']]


,entry_date_u6,experiment_id,match_position,mask_id_u6,unique_id_u6,mask_title_u6,mask_text_u6,mask_id_dev,unique_id_dev,mask_title_dev,mask_text_dev,mask_id_prod,unique_id_prod,mask_title_prod,mask_text_prod
649,2022-09-16,207225,3,8,row 1,Reaction temp,{REACTION_TEMP} °C,3,uid 5499,solvent volume,{SOLVENT_NAME} ({VOLUME} {VOLUME_UNITS}),8,row 1,Reaction temp,{REACTION_TEMP} °C
650,2022-09-16,207225,4,9,uid 1,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)",1071,uid 5500,solvent volume ELN,{SOLVENT_NAME}​​({VOLUME}​{VOLUME_UNITS})​,9,uid 1,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)"
651,2022-09-16,207225,1,9,uid 2,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)",1082,uid 1,Reactant volume or quant Test 1,"{REACTANT_NAME} (​{QUANT}, ​{MMOL} mmol)​",9,uid 2,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)"
652,2022-09-16,207225,2,9,uid 3,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)",1082,uid 2,Reactant volume or quant Test 1,"{REACTANT_NAME} (​{QUANT}, ​{MMOL} mmol)​",9,uid 3,reactant volume or quant,"{REACTANT_NAME} ({QUANT}, {MMOL} mmol)"


In [25]:
# Get user input for chunk size
try:
    user_input = input("Enter chunk size (integer) or a list of experiment IDs (comma-separated): ").strip()
    if "," in user_input:
        experiment_ids = [x.strip() for x in user_input.split(",") if x.strip().isdigit()]
        if not experiment_ids:
            raise ValueError("No valid experiment IDs provided.")
        user_selection = {"type": "filter", "experiment_ids": experiment_ids}
    elif user_input.isdigit():
        chunk_size = int(user_input)
        user_selection = {"type": "pagination", "chunk_size": chunk_size, "offset": 0}
    else:
        raise ValueError("Input must be a positive integer or a comma-separated list of integers.")
except ValueError as e:
    display(HTML(f"<strong style='color: red;'>Error: {e}</strong>"))

if user_selection["type"] == "pagination":
    display(HTML(f"<strong>Pagination mode: Chunk size set to {user_selection['chunk_size']} rows per page.</strong>"))
    if "pagination_state" not in globals():
        pagination_state = {"offset": user_selection["offset"], "chunk_size": user_selection["chunk_size"]}
    else:
        pagination_state["chunk_size"] = user_selection["chunk_size"]
        pagination_state["offset"] = 0
elif user_selection["type"] == "filter":
    display(HTML(f"<strong>Filter mode: Selected experiment IDs: {user_selection['experiment_ids']}</strong>"))
    offset = 0

Enter chunk size (integer) or a list of experiment IDs (comma-separated):  207423,207424,207425,207426,207427,207428,207429,207430,207431,207432,207433,207435,207442,207443,207447,207449,207454,207464,207465,215088,215187


In [30]:
def render_html(row, idx, ds_suffix):
    experiment_id = row.experiment_id
    entry_date = getattr(row, f'entry_date_prod')
    write_up = getattr(row, f'write_up_{ds_suffix}')
    return f"""
    <div style="border: 1px solid #ccc; margin: 10px 0; padding: 10px; background-color: #f9f9f9;">
        <strong>ExpID Group: {idx} - {ds_suffix.upper()}</strong>
        <div style="margin-top: 5px;">
            <em><strong>Experiment ID: </strong>{experiment_id}</em>
        </div>
        <div style="margin-top: 5px;">
            <em><strong>Entry Date: </strong>{entry_date}</em>
        </div>        
        <div style="border-top: 1px dashed #999; margin-top: 10px; padding-top: 10px">
            <span style="font-size: 18px !important;">{write_up}</span>
        </div>
    </div>
    """

if user_selection["type"] == "pagination":
    chunk_size = pagination_state["chunk_size"]
    offset = pagination_state["offset"]
    dff = merged_dff.iloc[offset:offset + chunk_size]
elif user_selection["type"] == "filter":
    dff = merged_dff[merged_dff.experiment_id.isin(user_selection["experiment_ids"])].sort_values(by="experiment_id", ascending=False)
    
for idx, row in enumerate(dff.itertuples(index=False), start=offset + 1):
    html_u6 = render_html(row, idx, 'u6') 
    html_dev = render_html(row, idx, 'dev') 
    html_prod = render_html(row, idx, 'prod') 
    display(HTML(html_dev))
    display(HTML(html_u6))
    display(HTML(html_prod))
    display(HTML('<hr style="border: 2px solid #999; margin: 30px 0; padding: 10px 0;">'))

if user_selection["type"] == "pagination":
    pagination_state["offset"] += chunk_size

    if pagination_state["offset"] >= len(merged_dff):
        pagination_state["offset"] = 0
        display(HTML("<strong>Restarting pagination: Beginning of dataset.</strong>"))